In [14]:
!pip install tensorflow_datasets

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_datasets as tfds
from Classifier import Classifier
from DataGenerator import DataGenerator

seed = 42

# prepare train, val, test datasets

In [2]:
dataset = pd.read_csv("../data/players_number/train_player_numbers.csv")
# update the path to the images
dataset["filepath"] = dataset["filepath"].apply(lambda x: "../data/players_number/"+x)

In [3]:
# number of samples varies for different numbers
print(dataset.groupby("label").agg("count"))
# smallest number of samples
min_count = dataset.groupby("label").agg("count").min()["filename"]
print(min_count)

       filename  video_frame  player  left   top  right  bottom  filepath
label                                                                    
0           130          130     130   130   130    130     130       130
1            70           70      70    70    70     70      70        70
2            99           99      99    99    99     99      99        99
3           110          110     110   110   110    110     110       110
4           116          116     116   116   116    116     116       116
...         ...          ...     ...   ...   ...    ...     ...       ...
95          552          552     552   552   552    552     552       552
96          633          633     633   633   633    633     633       633
97          838          838     838   838   838    838     838       838
98         1008         1008    1008  1008  1008   1008    1008      1008
99          735          735     735   735   735    735     735       735

[100 rows x 8 columns]
24


In [4]:
# randomly sample n samples with n=smallest number of samples 
subset = dataset.groupby("label").sample(n=min_count, replace=False, random_state=seed)

In [5]:
# one-hot encoding to use categorical cross entropy, not needed if sparse cat cross entropy
subset = pd.concat([subset, pd.get_dummies(subset["label"], prefix="n")], axis=1)

In [6]:
onehot_labels = [f"n_{i}" for i in range(100)]

In [7]:
train, test = train_test_split(subset, test_size=0.3, random_state=seed, shuffle=True)
test, val = train_test_split(test, test_size=0.3, random_state=seed, shuffle=True)

**make TF data generators**

In [11]:
DG = DataGenerator()

y_key = "label" #onehot_labels

train_ds = DG.get_dataset(
    filenames=train["filepath"].to_numpy(), 
    labels=train[y_key].to_numpy(), 
    batch_size=5, 
    n_prefetch=1,
    training=True
)

val_ds = DG.get_dataset(
    filenames=val["filepath"].to_numpy(), 
    labels=val[y_key].to_numpy())

test_ds = DG.get_dataset(
    filenames=test["filepath"].to_numpy(), 
    labels=test[y_key].to_numpy())

2022-01-03 10:21:56.860993: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**load double MNIST datasetto pretrain the network on a larger dataset**

https://github.com/shaohua0116/MultiDigitMNIST

In [8]:
def get_files(path, extensions=(".png", ".jpeg", ".jpg")):
    results = []
    for r, d, files in os.walk(path):
        for f in files:
            if f.endswith(extensions):
                results.append(os.path.join(r, f))
    return results

pngs = get_files(path="../data/double_mnist_seed_123_image_size_64_64", extensions=(".png"))
ds = []
for png in pngs:
    basename = os.path.splitext(os.path.basename(png))[0]
    label = basename.split("_")[-1]
    ds.append([png, int(label), int(label[0]), int(label[1])])
ds = pd.DataFrame(ds, columns=["path", "group", "digit_1", "digit_2"])

In [9]:
train = ds.groupby("group").sample(frac=0.9, replace=False)
test = ds.drop(index=train.index)

In [12]:
DG = DataGenerator()

y_key = "label" #onehot_labels

train_ds = DG.get_dataset(
    filenames=train["path"].to_numpy(), 
    labels=(train["digit_1"].to_numpy(), train["digit_2"].to_numpy()), 
    batch_size=32, 
    n_prefetch=1,
    training=True
)

val_ds = DG.get_dataset(
    filenames=test["path"].to_numpy(), 
    labels=(test["digit_1"].to_numpy(), test["digit_2"].to_numpy())
)


# Make classifier

In [ ]:
classifier = Classifier()
opt = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9)
classifier.model.compile(loss=["sparse_categorical_crossentropy", "sparse_categorical_crossentropy"],
    loss_weights=[0.5, 0.5],
    optimizer=opt, 
    metrics=["accuracy"])

# callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./model_weights/model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

classifier.model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/50
2813/2813 [==============================] - 157s 56ms/step - loss: 2.3035 - dense_2_loss: 2.3032 - dense_3_loss: 2.3039 - dense_2_accuracy: 0.1093 - dense_3_accuracy: 0.1138 - val_loss: 2.2877 - val_dense_2_loss: 2.2864 - val_dense_3_loss: 2.2890 - val_dense_2_accuracy: 0.1304 - val_dense_3_accuracy: 0.1367
Epoch 2/50
2813/2813 [==============================] - 134s 48ms/step - loss: 2.2751 - dense_2_loss: 2.2744 - dense_3_loss: 2.2759 - dense_2_accuracy: 0.1364 - dense_3_accuracy: 0.1417 - val_loss: 2.2493 - val_dense_2_loss: 2.2477 - val_dense_3_loss: 2.2509 - val_dense_2_accuracy: 0.2061 - val_dense_3_accuracy: 0.1941
Epoch 3/50
2813/2813 [==============================] - 135s 48ms/step - loss: 2.2074 - dense_2_loss: 2.2055 - dense_3_loss: 2.2092 - dense_2_accuracy: 0.1966 - dense_3_accuracy: 0.1964 - val_loss: 2.1188 - val_dense_2_loss: 2.1163 - val_dense_3_loss: 2.1214 - val_dense_2_accuracy: 0.2793 - val_dense_3_accuracy: 0.2747
Epoch 4/50
2813/2813 [===============

2813/2813 [==============================] - 136s 48ms/step - loss: 0.7047 - dense_2_loss: 0.7009 - dense_3_loss: 0.7085 - dense_2_accuracy: 0.7765 - dense_3_accuracy: 0.7726 - val_loss: 0.5272 - val_dense_2_loss: 0.5233 - val_dense_3_loss: 0.5311 - val_dense_2_accuracy: 0.8583 - val_dense_3_accuracy: 0.8510
Epoch 27/50
2813/2813 [==============================] - 138s 49ms/step - loss: 0.6902 - dense_2_loss: 0.6859 - dense_3_loss: 0.6945 - dense_2_accuracy: 0.7809 - dense_3_accuracy: 0.7773 - val_loss: 0.5130 - val_dense_2_loss: 0.5077 - val_dense_3_loss: 0.5183 - val_dense_2_accuracy: 0.8650 - val_dense_3_accuracy: 0.8554
Epoch 28/50
2813/2813 [==============================] - 134s 48ms/step - loss: 0.6720 - dense_2_loss: 0.6681 - dense_3_loss: 0.6759 - dense_2_accuracy: 0.7877 - dense_3_accuracy: 0.7847 - val_loss: 0.4950 - val_dense_2_loss: 0.4908 - val_dense_3_loss: 0.4991 - val_dense_2_accuracy: 0.8691 - val_dense_3_accuracy: 0.8602
Epoch 29/50
2813/2813 [=======================

# try transfer learning from xception pretrained on imagenet

classify first and second digit separately

In [89]:
# split the number into 2 digits
subset["first_digit"] = subset["label"].apply(lambda x: int(f'{x:02d}'[0]))
subset["second_digit"] = subset["label"].apply(lambda x: int(f'{x:02d}'[1]))


train, test = train_test_split(subset, test_size=0.3, random_state=seed, shuffle=True)
test, val = train_test_split(test, test_size=0.3, random_state=seed, shuffle=True)

In [101]:
DG = DataGenerator()

train_ds = DG.get_dataset(
    filenames=train["filepath"].to_numpy(), 
    labels=(train["first_digit"].to_numpy(), train["second_digit"].to_numpy()),
    batch_size=5, 
    n_prefetch=1,
    training=True
)

val_ds = DG.get_dataset(
    filenames=val["filepath"].to_numpy(), 
    labels=(val["first_digit"].to_numpy(), val["second_digit"].to_numpy()))

test_ds = DG.get_dataset(
    filenames=test["filepath"].to_numpy(), 
    labels=(test["first_digit"].to_numpy(), test["second_digit"].to_numpy()))

In [102]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
# base_modle = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False)

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
first_digit_output = tf.keras.layers.Dense(10, activation="softmax")(avg)
second_digit_output = tf.keras.layers.Dense(10, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=[first_digit_output, second_digit_output])

# freeze the weights
for layer in base_model.layers:
    layer.trainable = False

In [103]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(
    loss=["sparse_categorical_crossentropy", "sparse_categorical_crossentropy"],
    loss_weights=[0.5, 0.5],
    optimizer=optimizer, 
    metrics=["accuracy"]
)

In [104]:
history = model.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
336/336 [==============================] - 18s 48ms/step - loss: 2.4620 - dense_24_loss: 2.4265 - dense_25_loss: 2.4974 - dense_24_accuracy: 0.1423 - dense_25_accuracy: 0.1173 - val_loss: 2.4705 - val_dense_24_loss: 2.4083 - val_dense_25_loss: 2.5327 - val_dense_24_accuracy: 0.1713 - val_dense_25_accuracy: 0.1481
Epoch 2/5
336/336 [==============================] - 17s 51ms/step - loss: 2.1838 - dense_24_loss: 2.1465 - dense_25_loss: 2.2210 - dense_24_accuracy: 0.2351 - dense_25_accuracy: 0.2012 - val_loss: 2.3844 - val_dense_24_loss: 2.3225 - val_dense_25_loss: 2.4462 - val_dense_24_accuracy: 0.1574 - val_dense_25_accuracy: 0.1296
Epoch 3/5
336/336 [==============================] - 16s 48ms/step - loss: 2.0189 - dense_24_loss: 1.9891 - dense_25_loss: 2.0487 - dense_24_accuracy: 0.2958 - dense_25_accuracy: 0.2774 - val_loss: 2.3838 - val_dense_24_loss: 2.3241 - val_dense_25_loss: 2.4436 - val_dense_24_accuracy: 0.1991 - val_dense_25_accuracy: 0.1389
Epoch 4/5
336/336 [======